# Парсер статей с сайта Poisknews

Подключаем библиотеки

In [ ]:
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import re # Регулярные выражения.

Получаем информацию о загруженной веб-странице


In [ ]:
%%time
# %%time - Магия Jupyter - замеряет время выполнения ячейки. Должно быть первой строчкой в ячейке.
resp = requests.get("https://poisknews.ru/category/skript/")
print("cookies:", resp.cookies)
print("time to download:", resp.elapsed)
print("page encoding", resp.encoding)
print("Server response: ", resp.status_code)
print("Is everything ok? ", resp.ok)
print("Page's URL: ", resp.url)

cookies: <RequestsCookieJar[]>
time to download: 0:00:01.054683
page encoding UTF-8
Server response:  200
Is everything ok?  True
Page's URL:  https://poisknews.ru/category/skript/
CPU times: user 35.5 ms, sys: 7.83 ms, total: 43.4 ms
Wall time: 1.27 s


In [ ]:
# Получили объект BeautifulSoup и скормили ему текст страницы.
bs=BeautifulSoup(resp.text, "html5lib") 
# Вот таким образом можно попросить отдать первый тег, отмеченный как h1. Вместо h1 можно написать любой другой тег.
title=bs.h1.text
# Получаем все параграфы (тег p), берем их текст без тегов и склеиваем в один текст.
text=" ".join([p.text for p in bs.find_all("p")])
print(title, "\n-----\n", text)

Статьи 
-----
 Анализ пряди волос из могильника на испанском острове Менорка показал, что люди древней цивилизации использовали галлюциногены растительного происхождения. Это открытие – первое прямое указание на использование... Новая противораковая вакцина, подогнанная под генетические изменения в клетках опухоли отдельного пациента, показала многообещающие результаты в небольшом клиническом испытании, которое было проведено международной... Уезжают, приезжают, перемещаются… Добровольно, вынужденно… Понятия «эмиграция», «иммиграция», «миграция» прочно вошли в жизнь народов, стран и континентов. Наглядно познакомиться с современными трендами этих процессов... ПОИСК © 2023   
		Only fill in if you are not human
		
	  
 
		Only fill in if you are not human
		
	  
		Only fill in if you are not human
		
	


In [ ]:
# Теперь выделим только то, что взято в тег h2.
h2s=BeautifulSoup(requests.get("https://poisknews.ru/category/skript/").text, "html5lib").find_all("h2")
# Формируем список ссылок. Для этого берем первую (кстати, единственную) ссылку из каждого выделенного
# фрагмента, у нее берем значение параметра href. 
links=[l.find_all("a")[0]["href"] for l in h2s]
print(links)

['https://poisknews.ru/themes/arheologiya/s-podachi-shamanov-ritualy-s-upotrebleniem-gallyuczinogennyh-preparatov-provodilis-eshhe-v-bronzovom-veke/', 'https://poisknews.ru/themes/arheologiya/kuda-i-otkuda-o-migraczii-drevnih-lyudej-rasskazhut-ih-stoyanki/', 'https://poisknews.ru/themes/medicine/okonchatelno-i-bespovorotno-predotvratit-reczidiv-melanomy-pomozhet-personalizirovannaya-mrnkovaya-vakczina/', 'https://poisknews.ru/themes/space-themes/zhzhet-svetilo-nablyudeniya-kosmicheskogo-apparata-solar-orbiter-mogut-obyasnit-zapredelnuyu-temperaturu-solnechnoj-korony/', 'https://poisknews.ru/socziologiya/menyaetsya-obmen-unikalnyj-atlas-otrazil-proczessy-pereseleniya-lyudej-iz-strany-v-stranu-za-30-let/', 'https://poisknews.ru/themes/medicine/k-novoj-farmaczii-lekarstvo-ot-deficzita-lekarstv-iskali-na-zasedanii-kluba-mezhnauchnyh-kontaktov/', 'https://poisknews.ru/international/effekty-zamorozki-k-chemu-vedet-priostanovka-ustoyavshihsya-nauchnyh-svyazej/', 'https://poisknews.ru/skript/m

In [ ]:
#функция возвращающая список ссылок во всех тегах h2
def geth2(link):
  h2s=BeautifulSoup(requests.get(link).text, "html5lib").find_all("h2")
  links=[l.find_all("a")[0]["href"] for l in h2s]
  return links

In [ ]:
# Загрузка статьи по URL.
def getArticle(url):
    # Получает текст страницы.
    resp=requests.get(url)
    # Загружаем текст в объект типа BeautifulSoup.
    bs=BeautifulSoup(resp.text, "html5lib") 
    # Получаем заголовок статьи.
    aTitle=bs.h1.text.replace("\xa0", " ")

    #получаем текст статьи без последнего абзаца (автора)
    div = bs.find('div', class_='entry-content-text', itemprop='articleBody')
    paragraphs = div.find_all('p')

    # удаляем последний абзац
    if len(paragraphs) > 1:
      paragraphs = paragraphs[:-1]

    result_text = ""
    for paragraph in paragraphs:
      # заменяем неразрывные пробелы на обычные пробелы
      result_text += paragraph.text.replace('\xa0', ' ') + " "
    anArticle=result_text
    
    #получаем список категорий статьи
    categories=[]
    cat1=[]
    cat = bs.find_all('div', class_='entry-category')
    for l in cat:
      cat1=l.find_all('a')
      for b in cat1:
        text=b.text.strip()
        categories.append(text)

    # Возвращаем кортеж из заголовка, категорий и текста статьи.
    return aTitle, categories, anArticle 

In [ ]:
getArticle("https://poisknews.ru/international/effekty-zamorozki-k-chemu-vedet-priostanovka-ustoyavshihsya-nauchnyh-svyazej/")

('Эффекты заморозки. К чему ведет приостановка устоявшихся научных связей',
 ['Международное сотрудничество', 'Новости', 'В России', 'Статьи'],
 'На недавней XXIV Ясинской (Апрельской) конференции по проблемам развития экономики и общества, которая ежегодно проходит в НИУ ВШЭ, в числе других тем обсуждались и вопросы международного научного сотрудничества. Один из докладов был сделан признанным экспертом в этой области, давним автором «Поиска» доктором экономических наук, профессором НИУ ВШЭ Ириной ДЕЖИНОЙ. Мы решили обсудить эту тему и на страницах газеты. – Ирина, прошло больше года с тех пор, как исследовательские организации ведущих в научном отношении стран мира приостановили сотрудничество с российскими партнерами. В какой точке мы сегодня находимся? Какие меры по смягчению санкционного влияния оказались действенными? Какие сов\xadместные проекты удалось сохранить?\n– Если говорить о внешних факторах, то за прошедшее время ослабления санкционного давления на российскую науку не б

In [ ]:
import csv

In [ ]:
#создадим файл csv для сохранения новостей
headers = ['aTitle', 'cat', 'anArticle']

with open('news.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(headers)

In [ ]:
#функция записи в csv
def save_news_to_csv(urls_list, file_name): 

    # Открываем CSV-файл для записи новостей
    with open(file_name, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)


        # Записываем новости в CSV-файл
        for url in urls_list:
            article_data = getArticle(url)
            if article_data is not None:
                writer.writerow(article_data)

In [ ]:
#парсим первую страницу
link = "https://poisknews.ru/category/skript/"
print (link)
linklist = geth2(link)
save_news_to_csv(linklist, "news.csv")

https://poisknews.ru/category/skript/


In [ ]:
#и затем остальные, всего страниц 163, 
for i in range (2, 163):
  link = "https://poisknews.ru/category/skript/page/" + str(i) + "/"
  print (link)
  linklist = geth2(link)
  save_news_to_csv(linklist, "news.csv")
  


https://poisknews.ru/category/skript/page/2/
https://poisknews.ru/category/skript/page/3/
https://poisknews.ru/category/skript/page/4/
